In [73]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [74]:
import pymongo
from pymongo import MongoClient
import pandas as pd

In [75]:
client = MongoClient('localhost', 27017)

In [76]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [77]:
db = client.example

In [78]:
collection = db.guests

In [79]:
guests = pd.read_csv('/Users/mirelalynch/Desktop/Project 2 Launch  - Guests.csv')
rooms = pd.read_csv('/Users/mirelalynch/Desktop/Project 2 Launch  - Rooms.csv')
payments = pd.read_csv('/Users/mirelalynch/Desktop/Project 2 Launch  - Payments.csv')
reservations = pd.read_csv('/Users/mirelalynch/Desktop/Project 2 Launch  - Reservations.csv')

In [80]:
guests_dict = guests.to_dict('records')
rooms_dict = rooms.to_dict('records')
payments_dict = payments.to_dict('records')
reservations_dict = reservations.to_dict('records')


In [81]:
guests_collection = db.guests
rooms_collection = db.rooms
payments_collection = db.payments
reservations_collection = db.reservations

In [82]:
guests_collection.insert_many(guests_dict)
rooms_collection.insert_many(rooms_dict)
payments_collection.insert_many(payments_dict)
reservations_collection.insert_many(reservations_dict)


InsertManyResult([ObjectId('66573df7da63410c0d4ab81b'), ObjectId('66573df7da63410c0d4ab81c'), ObjectId('66573df7da63410c0d4ab81d'), ObjectId('66573df7da63410c0d4ab81e'), ObjectId('66573df7da63410c0d4ab81f'), ObjectId('66573df7da63410c0d4ab820'), ObjectId('66573df7da63410c0d4ab821'), ObjectId('66573df7da63410c0d4ab822'), ObjectId('66573df7da63410c0d4ab823'), ObjectId('66573df7da63410c0d4ab824'), ObjectId('66573df7da63410c0d4ab825'), ObjectId('66573df7da63410c0d4ab826'), ObjectId('66573df7da63410c0d4ab827'), ObjectId('66573df7da63410c0d4ab828'), ObjectId('66573df7da63410c0d4ab829'), ObjectId('66573df7da63410c0d4ab82a'), ObjectId('66573df7da63410c0d4ab82b'), ObjectId('66573df7da63410c0d4ab82c'), ObjectId('66573df7da63410c0d4ab82d'), ObjectId('66573df7da63410c0d4ab82e'), ObjectId('66573df7da63410c0d4ab82f'), ObjectId('66573df7da63410c0d4ab830'), ObjectId('66573df7da63410c0d4ab831'), ObjectId('66573df7da63410c0d4ab832'), ObjectId('66573df7da63410c0d4ab833'), ObjectId('66573df7da63410c0d4ab8

In [108]:
checkout_date = '05/31/24'

In [109]:
def clean_up_collection(collection):
    documents = list(collection.find())
    for doc in documents:
        cleaned_doc = {k.strip(): v.strip() if isinstance(v, str) else v for k, v in doc.items()}
        collection.update_one({'_id': doc['_id']}, {'$set': cleaned_doc})

In [110]:
clean_up_collection(reservations_collection)
clean_up_collection(guests_collection)

In [111]:
pipeline = [
    {
        '$match': {'Checkout_Date ': checkout_date}
    },
    {
        '$lookup': {
            'from': 'guests',
            'localField': 'Guest_ID ',
            'foreignField': 'Guest_ID',
            'as': 'guest_info'
        }
    },
    {
        '$unwind': '$guest_info'
    },
    {
        '$project': {
            'Guest_Name': '$guest_info.Name',
            '_id': 0
        }
    }
]


In [112]:
result = list(reservations_collection.aggregate(pipeline))

In [113]:
guests_df = pd.DataFrame(result)

In [114]:
guests_df

,Guest_Name
0,Liam Mitchell
1,Ella Russell
2,Isabella Hayes


""
